In [ ]:
import cv2 
import glob 
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Dropout, Dense, GlobalAveragePooling2D 
from keras.models import Model 
from keras.optimizers import Adam 
from numpy import expand_dims 
from keras.preprocessing import image 
import matplotlib.pyplot as plt 
import os 

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" 

# dimensions of our images. 
img_width, img_height = 299, 299 

train_data_dir = r'Brain vs Spine\Train'  # location of training data 
validation_data_dir = r'Brain vs Spine\VAL'  # location of validation data 

# number of samples used for determining the samples_per_epoch 
nb_train_samples = 60 
nb_validation_samples = 10 
epochs = 20 
batch_size = 5 

train_datagen = ImageDataGenerator( 
    rescale=1. / 255,  # normalize pixel values to [0,1] 
    shear_range=0.2, 
    zoom_range=0.2, 
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True
) 

val_datagen = ImageDataGenerator( 
    rescale=1. / 255  # normalize pixel values to [0,1] 
) 

train_generator = train_datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='binary'
) 

validation_generator = train_datagen.flow_from_directory( 
    validation_data_dir, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='binary'
) 

# Model definition
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)) 
model_top = Sequential() 
model_top.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:], data_format=None)) 
model_top.add(Dense(256, activation='relu')) 
model_top.add(Dropout(0.5)) 
model_top.add(Dense(1, activation='sigmoid')) 
model = Model(inputs=base_model.input, outputs=model_top(base_model.output)) 

model.compile(optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
              loss='binary_crossentropy', 
              metrics=['accuracy']
) 

# Training the model
history = model.fit_generator( 
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size, 
    epochs=epochs, 
    validation_data=validation_generator, 
    validation_steps=nb_validation_samples // batch_size
) 

print(history.history.keys()) 

# Plot results
plt.figure() 
plt.plot(history.history['accuracy'], 'orange', label='Training accuracy') 
plt.plot(history.history['val_accuracy'], 'blue', label='Validation accuracy') 
plt.plot(history.history['loss'], 'red', label='Training loss') 
plt.plot(history.history['val_loss'], 'green', label='Validation loss') 
plt.legend() 
plt.show() 

# Testing
image_paths = glob.glob(r"E:\Mishal\Brain vs Spine\Test\*.png") 
plt.ion() 
for imgpath in image_paths: 
    plt.figure() 
    img = cv2.imread(imgpath) 
    plt.clf() 
    plt.imshow(img) 
    plt.show() 
    plt.pause(2) 

    imgar = image.img_to_array(imgpath) 
    img1 = cv2.resize(img, (299, 299)) 
    x = expand_dims(img1, axis=0) * 1. / 255 
    score = model.predict(x) 
    print('Predicted:', score, 'Brain' if score < 0.5 else 'Spine')
